SDSS Cross-matching
----
Example of using astroquery to search the SDSS spectroscopy catalog and turn the result into a table with active links to the SDSS Object Explorer pages.

In [ ]:
import numpy as np
from astropy.table import Table
from astropy import coordinates as coords
from astropy import units as u

### Note on Requirements
Be sure that you have astroquery installed - that's an additional dependency for this tutorial beyond Astropy.  If you're following the instructions from the workshop you should have gotten it when you did an update.  But run this cell to check:

In [ ]:
from astroquery.sdss import SDSS

If that fails, you'll want to try this from a terminal (and you'll need to go to "Kernel->Restart" after doing it):
```
conda install -c astropy astroquery
```
or if you're not using conda, you'll need to do:
```
pip install astroquery
```

Define the functions that will do the work

In [ ]:
# Take a list of targets and search for them in the SDSS database. Add the spectroscopic IDs to the table.
def sdssquery(targets):
    specobjid = []
    for pos in targets['coords']:
        try: 
            xid = SDSS.query_region(pos,data_release=12,spectro=True,radius=10*u.arcsec)
            specobjid += [xid['specobjid'][0]]
        except:
            print("Not found: ",pos)
    targets['specobjid'] = specobjid
    return targets

# Create a URL to the Explorer from a spectroscopic ID.
def sdss_url(specobjid):
    urlbase = "http://skyserver.sdss.org/dr12/en/tools/explore/summary.aspx?"
    url = urlbase+"sid=%d" % (specobjid)
    href = '<a href=%s> sdss </a>' % url
    return href

# Run the two functions defined above for all the targets
def sdss(targets):
    targets = sdssquery(targets)
    targets['url'] = [sdss_url(sid) for sid in zip(targets['specobjid'])]
    return targets

For demonstration purposes, here is a table with three sources (known to be in the SDSS catalog). We're creating columns for ID, RA and Dec, and then we are adding a column that contains the coordinates as a SkyCoord object, which will be used for the catalog matching.

In [ ]:
targets = Table()
targets['ID'] = [133,229,222]
targets['RA'] = [158.43144,195.36813,192.14461]
targets['Dec'] = [63.888338,51.080956,12.567532]
targets['coords']=coords.SkyCoord(targets['RA'],targets['Dec'],unit='deg',frame='icrs')
targets

Now cross-match this catalog with the SDSS and output the table in notebook format, including a URL to the SDSS Object Explorer page for each target. We have to do a little editing of the HTML to get this to render in the notebook (although there may now be a better way to do this in astropy). 

In [ ]:
newtable = sdss(targets)
ipy_html = newtable.show_in_notebook()
# the show_in_notebook format has changed the HTML brackets to &lt and &gt; we need to change them back
ipy_html.data = ipy_html.data.replace('&lt;','<')
ipy_html.data = ipy_html.data.replace('&gt;','>')
ipy_html